In [2]:
import os
import pydicom
import numpy as np
from PIL import Image
import openslide as openslide
import scripts.conversion
from PIL.GimpGradientFile import linear
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
from IPython.display import clear_output


In [4]:
scripts.conversion.Conversiontopng.convert_dicom_svs_to_png(
    input_folder="data\\Yale_HER2_cohort\\SVS",
    output_folder="output",
    svs_level=0,
    auto_downsample=True,
    max_images=10
)

Found 192 files to process.


Converting files: 100%|██████████| 10/10 [00:02<00:00,  3.91file/s]


--- Conversion Summary ---
[SVS] Saved: Her2Neg_Case_02.svs (Time: 0.68s)
[SVS] Saved: Her2Neg_Case_08.svs (Time: 0.73s)
[SVS] Saved: Her2Neg_Case_07.svs (Time: 0.88s)
[SVS] Saved: Her2Neg_Case_05.svs (Time: 1.25s)
[SVS] Saved: Her2Neg_Case_04.svs (Time: 1.49s)
[SVS] Saved: Her2Neg_Case_06.svs (Time: 1.76s)
[SVS] Saved: Her2Neg_Case_03.svs (Time: 1.85s)
[SVS] Saved: Her2Neg_Case_10.svs (Time: 1.20s)
[SVS] Saved: Her2Neg_Case_09.svs (Time: 1.47s)
[SVS] Saved: Her2Neg_Case_01.svs (Time: 2.56s)
